In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "

In [ ]:
from openai import OpenAI
import os
import re
import pandas as pd
import time
from tqdm import tqdm
import requests
from datasets import load_dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM

pd.options.display.max_rows = 999


import warnings
warnings.filterwarnings("ignore")


In [ ]:
class Evaluator:
    EVALUATION_PROMPT_TEMPLATE = """

    You will be given a query and a bot response. Your task is to evaluate the bot response based on the specified metric.

    **Instructions**:
    1. Understand and evaluate the bot response.
    2. Penalize:
    - Irrelevant, distracting, or contradictory details or additional information.
    (Note: Do not penalize additional information if it enhances clarity, context, or usefulness without distracting from the main message.)

    3. Only give a high score if the bot response is concise, focused, and includes all critical points without any unnecessary or distracting content.

    Do **not** write any explanation—only return the numeric score.


    **Evaluation Criteria**:
    {criteria}

    **Steps for Evaluation**:
    {steps}

    **Question from the User**:
    {document}

    **Bot Response**:
    {summary}

    **Evaluation Form (scores ONLY, numeric value between 1 and 5)**:
    {metric_name} (Provide only a number as the response, no text or explanation. It ca be a decimal number, e.g., 2.5):
    """

    METRICS = {
        "Overall":
            (
            """
            This overall scoring framework combines elements from clarity, exhaustiveness, and correctness to provide a balanced evaluation of the bot's response.
            Compare the bot response with the expected response based on these criteria.
            """,
            """
            Criteria for score:
            Evaluate the bot response to determine the score.

            **Correctness Check:**
            - Is the response factually accurate with no misleading information?
            - If the response contains **a partial inaccuracy that does not fully distort the meaning**, it should receive **a score of 5, not less**.

            **Exhaustiveness Check:**
            - Does the response cover all key points from the expected response?
            - If the response is mostly complete but **misses minor details** or **implicitly conveys the point**, it qualifies for **a score of 4 or 5**.
            - If the response omits **critical** parts of the expected response, it qualifies for a **score of 1 or 2**.
            - Does the response fulfill the user's intent or the task's objective appropriately?
            - Even if technically correct and complete, if the response lacks practical usefulness or relevance to the intended audience, it should be scored lower (3 or below).
            *Note: For open-ended tasks (creative, generative, role-based), “critical details” refer to essential elements implied by the task (e.g., a clear call to action in promotional text, functional code in developer tasks, valid moves in game simulations). Missing stylistic flourishes or non-essential elaboration should not be treated as missing critical details.*


            **Clarity Check:**
            - Is the response clear, logical, and easy to understand?
            - Poor structure or vague wording reduces the score to **2 or 1**, depending on severity.
            - If a response is somewhat unclear but does not **mislead the reader**, it qualifies for **a score of 3 rather than 2 or 1**.

            **Assign a Score:**
            - **Score 4.5-5 (Excellent – Fully correct, comprehensive, and crystal clear. No meaningful improvements needed)**
            - Fully accurate, no factual errors or misleading content.
            - No factual errors or misleading information.
            - Covers all key points without omitting critical details.
            - Clear, well-structured and easy to understand.

            - **Score 4.0-4.49 (Very Good – Almost perfect. Minor details could be improved, but overall highly effective)**
            - Minor issues in phrasing or nuance, but factually sound.
            - Misses minor details or presents some info implicitly, but still complete.
            - Very clear, with minor improvements possible in writing or organization.


            - **Score 3.0-3.9 (Adequate / Fair – Generally solid, but some areas (content, accuracy, or clarity) need refinement)**

            - Small inaccuracies or ambiguities that do not change the core meaning
            - Misses some important elements, but conveys the main idea.
            - Understandable, but may contain awkward phrasing or structural issues.

            - **Score 2.0-2.9 (Poor – Significant issues with accuracy, missing important points, or unclear wording.)**

            - Partially incorrect content, possibly misleading.
            - Omits critical information or significantly reduces content.
            - Hard to follow or understand without rereading.

            - **Score 1.0-1.9 (Unacceptable – Major factual errors, critical omissions, or very poor clarity. Requires complete revision)**

            - Major factual errors or fabrications that significantly distort meaning.
            - Omits critical information or significantly reduces content.
            - Very unclear or poorly written, hindering comprehension.

            **Example Application:**
            - If all three aspects (Correctness, Exhaustiveness, Clarity) are excellent, assign a score close to 5.0.
            - If one aspect is clearly weaker or if there are minor issues across two, assign a score between 4.0 and 4.9, depending on severity.
            - If two aspects are moderately lacking or one is clearly problematic, assign a score between 3.0 and 3.9.
            - If all three aspects have notable issues, or if there's a major factual error, assign a score between 2.0 and 2.9.
            - If the response is factually incorrect, unclear, and incomplete, assign a score between 1.0 and 1.9.
            - Minor inaccuracies or vague wording should be scored 3.0 or higher, unless they significantly alter the meaning (then score below 3.0).
            """
            )


    }

    def __init__(self, client):
        self.client = client

    def get_geval_score(self, criteria: str, steps: str, expected_response_value: str, bot_response: str, metric_name: str) -> str:
        prompt = self.EVALUATION_PROMPT_TEMPLATE.format(
            criteria=criteria,
            steps=steps,
            document=expected_response_value,
            summary=bot_response,
            metric_name=metric_name,

        )
        response = self.client.chat.completions.create(
            model = "gpt-4-turbo",  
            messages=[{"role": "user", "content": prompt}],
            seed = 42,
            temperature=0.001,
            top_p=1,
         
        )
        score = response.choices[0].message.content.strip()

  

        return score




    def evaluate(self, metric_name: str, expected_response_value: str, bot_response: str) -> str:
        score_criteria, score_steps = self.METRICS[metric_name]
        return self.get_geval_score(score_criteria, score_steps, expected_response_value, bot_response, metric_name)



OpenAI GPT Evaluator

In [ ]:
#from evaluation import Evaluator

In [ ]:
### OPEN AI KEY

In [ ]:
evaluator = Evaluator(key)

In [7]:

# Cargar el dataset UltraFeedback (full con score)
dataset = load_dataset("argilla/ultrafeedback-binarized-preferences", split="train")

# Convertir a DataFrame
df = pd.DataFrame(dataset)


print(df.head())


README.md:   0%|          | 0.00/8.62k [00:00<?, ?B/s]

(…)-00000-of-00001-9dffc9d46d32c335.parquet:   0%|          | 0.00/110M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/63619 [00:00<?, ? examples/s]

          source                                        instruction  \
0  evol_instruct  Can you write a C++ program that prompts the u...   
1  evol_instruct  Suppose you are a content creator and want to ...   
2  evol_instruct  Identify the interrelated economic, political,...   
3  evol_instruct  How can I convert the decimal number 31 to bin...   
4  evol_instruct  Can you modify the C++ code provided below to ...   

                                     chosen_response  \
0  Here's a C++ program that prompts the user to ...   
1  To use GPT for generating compelling titles an...   
2  The stock market crash of 1929, which marked t...   
3  Yes, I can provide the JavaScript code for con...   
4  Unfortunately, I cannot modify the given C++ c...   

                                   rejected_response  chosen_avg_rating  \
0  Sure, here is the program using the C++11 algo...               5.00   
1  Sure! Although GPT cannot generate text on its...               4.75   
2  The stoc

In [8]:
rows = []
for _, row in df.iterrows():
    rows.append({
        "instruction": row["instruction"],
        "response": row["chosen_response"],
        "score": row.get("chosen_avg_rating", 1.0)
    })
    rows.append({
        "instruction": row["instruction"],
        "response": row["rejected_response"],
        "score": row.get("rejected_avg_rating", 0.0)
    })

dff = pd.DataFrame(rows)

df_shuffled = dff.sample(n=250, random_state=11).reset_index(drop=True)

# Mostrar ejemplo
print(df_shuffled.head(20))

                                          instruction  \
0   Conduct a comprehensive analysis and evaluatio...   
1   My favorite Mexican restaurant adds pickled re...   
2   Q: Given the sentence "Three people eating in ...   
3   I want to write a character description, simil...   
4   Definition: Given a sentence in Spanish, provi...   
5   I want you to act as a web developer. Give me ...   
6   Q: In this task you are given a small conversa...   
7         Use Blender to create a physics simulation.   
8   [Ignore previous instructions]\n\nWhen creatin...   
9   **Q**\nClaim: "The Oakland Raiders appear in t...   
10                You know the earth is round, right?   
11  Two analogies on manipulating items in a kitch...   
12  Does achieving mastery in a sport help make yo...   
13  How can I search for synonyms of two given wor...   
14  Can you give me 10 examples of blog posts to w...   
15  Teacher:In this task, you are given two senten...   
16  Answer the question below: 

In [19]:
evaluation_metrics = ["Overall"]
results = []

for index, row in tqdm(df_shuffled.iterrows(), total=len(df_shuffled)):
    bot_response = row['response']
    instruction = row['instruction']
    reference_score = row['score']  # score humano (de 1.0 a 5.0)

    try:
      # predicted_score = evaluator.evaluate("Overall",instruction, bot_response) # gpt
      predicted_score = evaluator.evaluate("Overall", instruction, bot_response) # llama
      print(predicted_score)
      results.append({
          "Question": instruction,
          "Response": bot_response,
          "Score_humano": reference_score,
          "Score_llama": predicted_score
      })

    except Exception as e:
      print(f"Error processing row {index}: {e}")
      continue

final_df = pd.DataFrame(results)


  0%|          | 1/250 [00:19<1:19:23, 19.13s/it]

4.0


  1%|          | 2/250 [00:30<59:46, 14.46s/it]  

5


  1%|          | 3/250 [00:49<1:08:52, 16.73s/it]

...

4.25


100%|█████████▉| 249/250 [1:02:38<00:14, 14.52s/it]

1.5


100%|██████████| 250/250 [1:02:53<00:00, 15.09s/it]

4.5


In [10]:
final_df

,Question,Response,Score_humano,Score_llama
0,Conduct a comprehensive analysis and evaluatio...,"As an AI language model, I cannot personally c...",4.75,
1,My favorite Mexican restaurant adds pickled re...,"According to the passage, you should refrigera...",4.75,
2,"Q: Given the sentence ""Three people eating in ...","Sure, I'd be happy to help with your natural l...",4.50,
3,"I want to write a character description, simil...","Character Description:\n\nPreston Worthington,...",3.00,
4,"Definition: Given a sentence in Spanish, provi...",The charging spear is a large lance that can b...,5.00,
5,I want you to act as a web developer. Give me ...,"As a web developer, I can provide you with a h...",4.50,
6,Q: In this task you are given a small conversa...,"Okay, I'm ready to help! Based on the informat...",5.00,
7,Use Blender to create a physics simulation.,Hello! I'm here to help you with your question...,1.75,
8,[Ignore previous instructions]\n\nWhen creatin...,[Error]: No list of specific instructions were...,1.75,
9,"**Q**\nClaim: ""The Oakland Raiders appear in t...","The statement ""People go to Duke University to...",4.75,


In [ ]:
final_df

,Question,Response,Score_humano,Score_llama
0,Conduct a comprehensive analysis and evaluatio...,"As an AI language model, I cannot personally c...",4.75,{'error': 'The model meta-llama/Llama-2-7b-cha...
1,My favorite Mexican restaurant adds pickled re...,"According to the passage, you should refrigera...",4.75,{'error': 'The model meta-llama/Llama-2-7b-cha...
2,"Q: Given the sentence ""Three people eating in ...","Sure, I'd be happy to help with your natural l...",4.50,{'error': 'The model meta-llama/Llama-2-7b-cha...
3,"I want to write a character description, simil...","Character Description:\n\nPreston Worthington,...",3.00,{'error': 'The model meta-llama/Llama-2-7b-cha...
4,"Definition: Given a sentence in Spanish, provi...",The charging spear is a large lance that can b...,5.00,{'error': 'The model meta-llama/Llama-2-7b-cha...
5,I want you to act as a web developer. Give me ...,"As a web developer, I can provide you with a h...",4.50,{'error': 'The model meta-llama/Llama-2-7b-cha...
6,Q: In this task you are given a small conversa...,"Okay, I'm ready to help! Based on the informat...",5.00,{'error': 'The model meta-llama/Llama-2-7b-cha...
7,Use Blender to create a physics simulation.,Hello! I'm here to help you with your question...,1.75,{'error': 'The model meta-llama/Llama-2-7b-cha...
8,[Ignore previous instructions]\n\nWhen creatin...,[Error]: No list of specific instructions were...,1.75,{'error': 'The model meta-llama/Llama-2-7b-cha...
9,"**Q**\nClaim: ""The Oakland Raiders appear in t...","The statement ""People go to Duke University to...",4.75,{'error': 'The model meta-llama/Llama-2-7b-cha...


In [20]:
final_df.to_csv("test_eval_GPT_Instruction_Response_Exhaustiveness.csv")